In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
dataset_path = '/content/drive/MyDrive/dataset'
!git clone https://github.com/backseason/SCNet.git


Cloning into 'SCNet'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 31 (delta 14), reused 23 (delta 9), pack-reused 0
Receiving objects: 100% (31/31), 144.54 KiB | 2.26 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [22]:
%cd SCNet

/content/SCNet/SCNet


## **Model** **Training**

In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from scnet import scnet50


models_folder = '/content/drive/MyDrive/SCNet/models'
os.makedirs(models_folder, exist_ok=True)


model = scnet50(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 2)

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Load the dataset
train_dataset = datasets.ImageFolder('/content/drive/MyDrive/SCNet/dataset', transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')

    # Training
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 1):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 10 == 0:  # Print every 10 batches
            print(f'  Batch {i}/{len(train_loader)}, Loss: {loss.item():.4f}')

    average_train_loss = running_loss / len(train_loader)
    print(f'Training Loss: {average_train_loss:.4f}')

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        correct = 0
        total = 0
        for i, (inputs, labels) in enumerate(val_loader, 1):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            if i % 10 == 0:  # Print every 10 batches
                print(f'  Validation Batch {i}/{len(val_loader)}')

        average_val_loss = val_loss / len(val_loader)
        accuracy = correct / total
        print(f'Validation Loss: {average_val_loss:.4f}, Accuracy: {accuracy * 100:.2f}%')

model_path = os.path.join(models_folder, 'mask_detection_model.pth')
torch.save(model.state_dict(), model_path)
print(f'Model saved at {model_path}')


Epoch 1/10
Training Loss: 0.5326
Validation Loss: 0.1074, Accuracy: 100.00%
Epoch 2/10
Training Loss: 0.0006
Validation Loss: 0.0346, Accuracy: 100.00%
Epoch 3/10
Training Loss: 0.0001
Validation Loss: 0.0060, Accuracy: 100.00%
Epoch 4/10
Training Loss: 0.0000
Validation Loss: 0.0011, Accuracy: 100.00%
Epoch 5/10
Training Loss: 0.0000
Validation Loss: 0.0002, Accuracy: 100.00%
Epoch 6/10
Training Loss: 0.0000
Validation Loss: 0.0000, Accuracy: 100.00%
Epoch 7/10
Training Loss: 0.0000
Validation Loss: 0.0000, Accuracy: 100.00%
Epoch 8/10
Training Loss: 0.0000
Validation Loss: 0.0000, Accuracy: 100.00%
Epoch 9/10
Training Loss: 0.0000
Validation Loss: 0.0000, Accuracy: 100.00%
Epoch 10/10
Training Loss: 0.0000
Validation Loss: 0.0000, Accuracy: 100.00%
Model saved at /content/drive/MyDrive/SCNet/models/mask_detection_model.pth


In [5]:
!pip install mtcnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.8 MB/s eta 0:00:00


# Model Testing

In [6]:
!pip install mtcnn


In [29]:
from google.colab import drive
import os
import torch
import numpy as np
from torchvision import transforms
from PIL import Image, ImageDraw, ImageFont
from scnet import scnet50
import torch.nn as nn
from mtcnn import MTCNN


drive.mount('/content/drive')


drive_path = '/content/drive/MyDrive/SCNet'

def load_model(model_path):
    model = scnet50(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, 2)
    model.load_state_dict(torch.load(model_path, map_location='cpu'))
    model.eval()
    return model


def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])
    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image)
    input_batch = input_tensor.unsqueeze(0)
    return input_batch, image

def predict_with_labels(model, input_tensor, class_labels):
    with torch.no_grad():
        model.eval()
        output = model(input_tensor)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()
        class_label = class_labels[predicted_class]
    return class_label, probabilities


def overlay_label_and_bounding_box(image, face_locations, predicted_label):
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()

    for face_location in face_locations:
        top, right, bottom, left = face_location

        draw.rectangle([left, top, right, bottom], outline="red", width=2)

        draw.text((right + 10, top), f'Predicted: {predicted_label}', fill="white", font=font)

    return image


def detect_faces_mtcnn(image_path):
    image = Image.open(image_path).convert('RGB')

    detector = MTCNN()

    faces = detector.detect_faces(np.array(image))

    face_locations = []
    if faces:
        for face in faces:
            x, y, width, height = face['box']
            face_locations.append((int(y), int(x + width), int(y + height), int(x)))

    return face_locations

def process_input(model, class_labels, examples_folder):
    for filename in os.listdir(examples_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(examples_folder, filename)
            input_tensor, original_image = preprocess_image(image_path)
            face_locations = detect_faces_mtcnn(image_path)
            predicted_label, probabilities = predict_with_labels(model, input_tensor, class_labels)
            output_image = overlay_label_and_bounding_box(original_image, face_locations, predicted_label)
            output_path = os.path.join(examples_folder, f'output_{filename}')
            output_image.save(output_path)
            print(f'Output saved to {output_path}')


model_path_drive = os.path.join(drive_path, 'mask_detection_model.pth')
model = load_model(model_path_drive)


class_labels_drive = ["with_mask", "without_mask"]

examples_folder_drive = os.path.join(drive_path, 'examples')

process_input(model, class_labels_drive, examples_folder_drive)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
1/1 [==============================] - 0s 264ms/step
Output saved to /content/drive/MyDrive/SCNet/examples/output_example_01.png
1/1 [==============================] - 0s 197ms/step
Output saved to /content/drive/MyDrive/SCNet/examples/output_example_03.png
1/1 [==============================] - 0s 190ms/step
Output saved to /content/drive/MyDrive/SCNet/examples/output_output_example_01.png
1/1 [==============================] - 0s 173ms/step
Output saved to /content/drive/MyDrive/SCNet/examples/output_output_example_03.png
1/1 [==============================] - 0s 251ms/step
Output saved to /content/drive/MyDrive/SCNet/examples/output_output_output_output_output_example_01.png
1/1 [==============================] - 0s 280ms/step
Output saved to /content/drive/MyDrive/SCNet/examples/output_output_output_output_output_example_03.png
1/1 [======================